In [6]:
from matplotlib import pyplot as plt
from pathlib import Path
from sklearn.metrics import mean_squared_error
from torch.utils.data import DataLoader

import numpy as np
import progressbar
import torch
import xarray as xr


from data.satellite import SatelliteData
import utils

%matplotlib inline 

Band info for reference: https://docs.sentinel-hub.com/api/latest/data/sentinel-2-l1c/

In [7]:
kwargs = {
    "data_root": "../Arizona-processed2",
    "seq_len": 60,
    "no_randomization": True
}
train_data = SatelliteData(train=True, **kwargs)
test_data = SatelliteData(train=False, **kwargs)

Using: 6510 for training
Using: 2790 for testing


In [20]:
data_threads = 5
batch_size = 1

train_loader = DataLoader(train_data,
                          num_workers=data_threads,
                          batch_size=batch_size,
                          shuffle=True,
                          drop_last=True,
                          pin_memory=True)
test_loader = DataLoader(test_data,
                         num_workers=data_threads,
                         batch_size=batch_size,
                         shuffle=True,
                         drop_last=True,
                         pin_memory=True)

In [21]:
dtype = torch.cuda.FloatTensor

def get_training_batch():
    while True:
        for sequence in train_loader:
            batch = utils.normalize_data("", dtype, sequence)
            yield batch
            
def get_testing_batch():
    while True:
        for sequence in test_loader:
            batch = utils.normalize_data("", dtype, sequence)
            yield batch 
            

training_batch_generator = get_training_batch()
testing_batch_generator = get_testing_batch()

In [22]:
class LinearRegression(torch.nn.Module):
    def __init__(self, inputSize, outputSize):
        super(LinearRegression, self).__init__()
        self.linear = torch.nn.Linear(inputSize, outputSize)

    def forward(self, x):
        out = self.linear(x)
        return out

In [23]:
img_size = 64
bands = 12

inputDim = 53*batch_size*bands        # takes variable 'x' 
outputDim = 7*batch_size*bands      # takes variable 'y'
learningRate = 0.01 
epochs = 100

model = LinearRegression(inputDim, outputDim)
##### For GPU #######
if torch.cuda.is_available():
    print("cuda available")
    model.cuda()

RuntimeError: [enforce fail at CPUAllocator.cpp:64] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 3585223950336 bytes. Error code 12 (Cannot allocate memory)


In [12]:
x = next(training_batch_generator)

In [13]:
x_train = torch.stack(x[:-7])

In [14]:
x_train.shape

torch.Size([53, 10, 12, 64, 64])

In [15]:
xx = torch.flatten(x_train)

In [16]:
xx.shape

torch.Size([26050560])

In [22]:
criterion = torch.nn.MSELoss() 
optimizer = torch.optim.SGD(model.parameters(), lr=learningRate)

In [49]:
for epoch in range(epochs):
    
    data = next(training_batch_generator)
    x_train = torch.stack(x[:-7])
    y_train = torch.stack(x[-7:])
    
    # Converting inputs and labels to Variable
    
    inputs = x_train
    labels = y_train
    
    if torch.cuda.is_available():
        inputs = inputs.cuda()
        labels = labels.cuda()

    # Clear gradient buffers because we don't want any gradient from previous epoch to carry forward, dont want to cummulate gradients
    optimizer.zero_grad()

    # get output from the model, given the inputs
    outputs = model(inputs)

    # get loss for the predicted output
    loss = criterion(outputs, labels)
    print(loss)
    # get gradients w.r.t to parameters
    loss.backward()

    # update parameters
    optimizer.step()

    print('epoch {}, loss {}'.format(epoch, loss.item()))

RuntimeError: size mismatch, m1: [407040 x 64], m2: [1 x 1] at /opt/conda/conda-bld/pytorch_1565272271120/work/aten/src/THC/generic/THCTensorMathBlas.cu:273